In [18]:
import pandas as pd 
import numpy as np
from torch.utils.data import dataset, DataLoader 
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import gradio as gr 
import torch 

In [ ]:
import pandas as pd 
import numpy as np
from torch.utils.data import dataset, DataLoader 
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import gradio as gr 
import torch 

In [19]:
torch.set_default_dtype(torch.float64)

In [20]:
df = pd.read_csv("CVD_cleaned.csv")

In [21]:
features = df.loc[:, ~df.columns.isin(["Heart_Disease"])]
targets = df.loc[:,df.columns.isin(["Heart_Disease"])]

In [22]:
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, LabelBinarizer
from sklearn.compose import ColumnTransformer

class MyDataset(Dataset):
    def __init__(self, features, targets, transform=None):
        self.raw_features = features 
        self.raw_targets = targets
        
        # Perform one-hot encoding on the features
        self.column_transformer = ColumnTransformer([('one_hot_encoder', OneHotEncoder(sparse=False), 
                                                        self.raw_features.select_dtypes(include=['object']).columns)],
                                                        remainder='passthrough')
        self.new_features = self.column_transformer.fit_transform(self.raw_features)
        self.label_binarizer = LabelBinarizer()
        self.new_targets =  self.label_binarizer.fit_transform(self.raw_targets.iloc[:,0])
        self.transform = transform 
        # self.dataloader = data_loader
        
    def __len__(self):
        return len(self.raw_targets)
    
    def __getitem__(self, index):
        sample = self.new_features[index], self.new_targets[index]
        if self.transform is not None:
            sample = self.transform(sample)
        return sample
    
    # def data_loader(self, batch_size=32, shuffle=True):
    #     # Create a DataLoader for the current dataset instance
    #     return DataLoader(dataset=self, batch_size=batch_size, shuffle=shuffle)
    
class to_tensor:
    def __call__(self,sample):
        features, targets= sample
        return torch.tensor(features), torch.tensor(targets)
    
def transformInput(dataloader:"dataloader",df:"dataframe"):
    return torch.tensor(dataloader.dataset.column_transformer.transform(df),dtype=torch.float64)
    

In [23]:
# Example usage:
dataset = MyDataset(features, targets, transform=to_tensor())
dataloader = DataLoader(dataset, batch_size=100, shuffle=True)

In [25]:
import torch.nn as nn
# number of features (len of X cols)
input_dim = 48
# number of hidden layers
hidden_layers = 25
# number of classes (unique of y)
output_dim = 2
class Network(nn.Module):
  def __init__(self):
    super(Network, self).__init__()
    self.linear1 = nn.Linear(input_dim, hidden_layers)
    self.linear2 = nn.Linear(hidden_layers, output_dim)
  def forward(self, x):
    x = torch.sigmoid(self.linear1(x))
    x = self.linear2(x)
    return x

In [26]:
clf = Network()

In [27]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(clf.parameters(), lr=0.1)

In [ ]:
epochs = 100
for epoch in range(epochs):
  running_loss = 0.0
  for i, data in enumerate(dataloader, 0):
    inputs, labels = data
    # set optimizer to zero grad to remove previous epoch gradients
    optimizer.zero_grad()
    # forward propagation
    outputs = clf(inputs)
    prediction = torch.softmax(outputs,dim=1)
    new_labels = torch.argmax(labels,dim=1)
    loss = criterion(outputs, new_labels)
    # backward propagation
    loss.backward()
    # optimize
    optimizer.step()
    running_loss += loss.item()
  # display statistics
  # print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.5f}')

In [34]:
def predict(
            General_Health, Checkup, Exercise, Skin_Cancer, Other_Cancer,
            Depression, Diabetes, Arthritis, Sex, Age_Category, Height_cm,Weight_kg, 
            BMI, Smoking_History,Alcohol_Consumption, Fruit_Consumption,
            Green_Vegetables_Consumption, FriedPotato_Consumption):
    
    df = pd.DataFrame.from_dict(
        {
            'General_Health' : [General_Health],
            'Checkup' : [Checkup], 
            'Exercise' : [Exercise], 
            'Skin_Cancer' : [Skin_Cancer], 
            'Other_Cancer' : [Other_Cancer],
            'Depression' : [Depression], 
            'Diabetes' : [Diabetes],
            'Arthritis' : [Arthritis], 
            'Sex' : [Sex], 
            'Age_Category' : [Age_Category],
            'Height_(cm)': [Height_cm], 
            'Weight_(kg)' : [Weight_kg], 
            'BMI' : [BMI], 
            'Smoking_History' : [Smoking_History],
            'Alcohol_Consumption' : [Alcohol_Consumption], 
            'Fruit_Consumption' : [Fruit_Consumption],
            'Green_Vegetables_Consumption' : [Green_Vegetables_Consumption], 
            'FriedPotato_Consumption' : [FriedPotato_Consumption]     
        }
    )
    
    data_df = transformInput(dataloader,df)
    with torch.no_grad():
        pred = clf(data_df)
        probs = nn.functional.softmax(pred)
        # pred_class = torch.argmax(probs, dim=1).tolist()
        # pred_class = dataloader.dataset.label_binarizer.inverse_transform(probs)
        #  tuple(predicted_class.tolist())
        # pred_class = dataloader.dataset.label_binarizer.inverse_transform(pred_class)
    # 
    return {"No Heart Disease": format(float(probs[0][0]), 'f'), "Yes Heart Disease": format(float(probs[0][1]), 'f')} 


demo = gr.Interface(
    predict,
    
    
    [
        gr.Dropdown(['Poor', 'Very Good', 'Good', 'Fair', 'Excellent']),
        gr.Dropdown(['Within the past 2 years', 'Within the past year','5 or more years ago','Within the past 5 years','Never']),
        gr.Dropdown(['No', 'Yes']),
        gr.Dropdown(['No', 'Yes']),
        gr.Dropdown(['No', 'Yes']),
        gr.Dropdown(['No', 'Yes']),
        gr.Dropdown(['No','Yes','No, pre-diabetes or borderline diabetes','Yes, but female told only during pregnancy']),
        gr.Dropdown(['Yes', 'No']),
        gr.Dropdown(['Female', 'Male']),
        gr.Dropdown(['70-74', '60-64', '75-79',   '80+', '65-69', '50-54', '45-49', '18-24','30-34', '55-59', '35-39', '40-44', '25-29']),
        gr.Slider(60, 280),
        gr.Slider(20, 300),
        gr.Slider(10, 100),
        gr.Dropdown(['No', 'Yes']),
        gr.Slider(0, 30),
        gr.Slider(0, 120),
        gr.Slider(0, 130),
        gr.Slider(0, 130),
        ],
    "label",
    interpretation="default",
    live=True,
    examples=[
        ['Very Good','Within the past year','No','No','No','No','Yes','No','Female','70-74',165.0,77.11,28.29,'No',0.0,30.0,0.0,4.0],
        ['Fair','5 or more years ago','Yes','Yes','Yes','No','Yes','No','Male','70-74',155.0,77.11,28.29,'No',0.0,30.0,0.0,4.0],
    ],
)

if __name__ == "__main__":
    demo.launch()


/Users/tonderaimadamba/Library/Python/3.9/lib/python/site-packages/gradio/blocks.py:275: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(f"api_name {api_name} already exists, using {api_name_}")
/Users/tonderaimadamba/Library/Python/3.9/lib/python/site-packages/gradio/blocks.py:275: UserWarning: api_name predict already exists, using predict_2
  warnings.warn(f"api_name {api_name} already exists, using {api_name_}")
/Users/tonderaimadamba/Library/Python/3.9/lib/python/site-packages/gradio/blocks.py:275: UserWarning: api_name predict already exists, using predict_3
  warnings.warn(f"api_name {api_name} already exists, using {api_name_}")
/Users/tonderaimadamba/Library/Python/3.9/lib/python/site-packages/gradio/blocks.py:275: UserWarning: api_name predict already exists, using predict_4
  warnings.warn(f"api_name {api_name} already exists, using {api_name_}")
/Users/tonderaimadamba/Library/Python/3.9/lib/python/site-packages/gradio/blocks.py:275: UserWarning

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
